In [1]:
#import modules
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_wine
import random
import time


In [2]:
#Carregando dados 

data = load_wine()
X = data.data
y = data.target
np.size(y,0)

178

In [3]:
#Treinamento e teste
#test_size = 0.3
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
#scaler = StandardScaler()
#scaler.fit(X_train)
#X_train = scaler.transform(X_train)
#X_test = scaler.transform(X_test)


In [4]:
##Inicialização das soluções
xlsfile = pd.ExcelFile('X_train.xlsx')
X_train = xlsfile.parse('Sheet1')
X_train = X_train[X_train.columns[1:]]
X_train = X_train.to_numpy()
X_train

xlsfile1 = pd.ExcelFile('X_test.xlsx')
X_test = xlsfile1.parse('Sheet1')
X_test = X_test[X_test.columns[1:]]
X_test = X_test.to_numpy()
X_test


xlsfile2 = pd.ExcelFile('y_train.xlsx')
y_train = xlsfile2.parse('Sheet1')
y_train = y_train[y_train.columns[1:]]
#y_train = y_train.to_numpy()
y_train=y_train.values.flatten()


xlsfile3 = pd.ExcelFile('y_test.xlsx')
y_test = xlsfile3.parse('Sheet1')
y_test = y_test[y_test.columns[1:]]
#y_test = y_test.to_numpy()
y_test = y_test.values.flatten()

In [5]:
#Arquitetura da rede
n_inputs = 13 # features
n_hidden = 20 
n_classes = 3
num_samples = X_train.shape[0] 
num_samples

124

In [6]:
def logits_function(p,X_train):
    """ Calculate roll-back the weights and biases

    Inputs
    ------
    p: np.ndarray
        The dimensions should include an unrolled version of the
        weights and biases.

    Returns
    -------
    numpy.ndarray of logits for layer 2

    """
    #coeficientes
    c1 = n_hidden*n_inputs
    c2 = c1 +n_hidden
    c3 = c2 + (n_hidden*n_hidden)
    c4=  c3 + n_hidden
    c5 = c4+(n_hidden*n_classes)
    
     # First layer weights
    W1 = p[0:c1].reshape(n_inputs, n_hidden) - 1
    
    # First layer bias
    b1 = p[c1:c2].reshape(( n_hidden,))
    
    # Second layer weights
    W2 = p[c2:c3].reshape(n_hidden, n_hidden) - 1
    
    # Second layer bias
    b2 = p[c3:c4].reshape(( n_hidden,))
    W3 = p[c4:c5].reshape(n_hidden, n_classes) - 1
    b3 = p[c5:c5+n_classes].reshape((n_classes,))
    

    # Perform forward propagation
    z1 = X_train.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    
    # Second linear step
    z2 = a1.dot(W2) + b2
    
    # Second activation function
    a2 = np.tanh(z2)
    
    #Third linear step
    z3 = a2.dot(W3) + b3
    
    #For the Third linear activation function we use the softmax function, either the sigmoid of softmax should be used for the last layer
    logits = z3
   
      
    
    
    
    return logits          # Logits for Layer 2

In [7]:
# Forward propagation
def forward_prop(params):
    """Forward propagation as objective function

    This computes for the forward propagation of the neural network, as
    well as the loss.

    Inputs
    ------
    params: np.ndarray
        The dimensions should include an unrolled version of the
        weights and biases.

    Returns
    -------
    float
        The computed negative log-likelihood loss given the parameters
    """

    logits = logits_function(params,X_train)

    # Compute for the softmax of the logits
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Compute for the negative log likelihood

    corect_logprobs = -np.log(probs[range(num_samples), y_train])
    loss = np.sum(corect_logprobs) / num_samples
    #print(loss)
    return loss

In [8]:
def f(x):
    
    
    
    
    """Higher-level method to do forward_prop in the
    whole swarm.

    Inputs
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
        The swarm that will perform the search

    Returns
    -------
    numpy.ndarray of shape (n_particles, )
        The computed loss for each particle
    """
    
    n_particles = x.shape[0]
    j = forward_prop(x)
    #print(x)
    return np.array(j)

In [9]:
##Inicialização das soluções
#lsfile = pd.ExcelFile('solucao_inicial.xlsx')
#ol = xlsfile.parse('Sheet1')
#ol = sol[sol.columns[1:]]
#nitpos = sol.to_numpy()
# = initpos[0:30]


In [10]:
class GWO:
     def __init__(self,n_agents,max_iters,dim,lb,up):
         
        self.lb = lb
        self.ub = up
        self.max_ite = max_iters
        self.dim = dim
        self.SearchAgents_no = n_agents
        self.Positions = np.zeros((n_agents, dim)) #lista referente a populacao corrente
        self.hist = []#lista referente ao historico
        # initialize alpha, beta, and delta_pos
        self.Alpha_pos = np.zeros(dim)
        self.Alpha_score = float("inf")
        
        self.Beta_pos = np.zeros(dim)
        self.Beta_score = float("inf")

        self.Delta_pos = np.zeros(dim)
        self.Delta_score = float("inf")
        
     def gerar_populacao(self):
        xlsfile = pd.ExcelFile('solucao_inicial.xlsx')
        sol = xlsfile.parse('Sheet1')
        sol = sol[sol.columns[1:]]
        initpos = sol.to_numpy()
        self.Positions = initpos[0:30]
        return
    
     def calculateFitness(self):
        for i in range(0, self.SearchAgents_no):
            for j in range(self.dim):
                self.Positions[i, j] = np.clip(self.Positions[i, j], self.lb[j], self.ub[j])
            
            fitness = f(self.Positions[i,:])
            if fitness < self.Alpha_score:
                self.Delta_score = self.Beta_score  # Update delte
                self.Delta_pos = self.Beta_pos.copy()
                self.Beta_score = self.Alpha_score  # Update beta
                self.Beta_pos = self.Alpha_pos.copy()
                self.Alpha_score = fitness
                # Update alpha
                self.Alpha_pos = self.Positions[i, :].copy()

            if fitness > self.Alpha_score and fitness < self.Beta_score:
                self.Delta_score = self.Beta_score  # Update delte
                self.Delta_pos = self.Beta_pos.copy()
                self.Beta_score = fitness  # Update beta
                self.Beta_pos = self.Positions[i, :].copy()

            if fitness > self.Alpha_score and fitness > self.Beta_score and fitness < self.Delta_score:
                self.Delta_score = fitness  # Update delta
                self.Delta_pos = self.Positions[i, :].copy()
        return
    
     def updateWolves(self,a):
        for i in range(0, self.SearchAgents_no):
            for j in range(0, self.dim):
                r1 = random.random()  # r1 is a random number in [0,1]
                r2 = random.random()  # r2 is a random number in [0,1]

                A1 = 2 * a * r1 - a
                # Equation (3.3)
                C1 = 2 * r2
                # Equation (3.4)

                D_alpha = abs(C1 * self.Alpha_pos[j] - self.Positions[i, j])
                # Equation (3.5)-part 1
                X1 = self.Alpha_pos[j] - A1 * D_alpha
                # Equation (3.6)-part 1

                r1 = random.random()
                r2 = random.random()

                A2 = 2 * a * r1 - a
                # Equation (3.3)
                C2 = 2 * r2
                # Equation (3.4)

                D_beta = abs(C2 * self.Beta_pos[j] - self.Positions[i, j])
                # Equation (3.5)-part 2
                X2 = self.Beta_pos[j] - A2 * D_beta
                # Equation (3.6)-part 2

                r1 = random.random()
                r2 = random.random()

                A3 = 2 * a * r1 - a
                # Equation (3.3)
                C3 = 2 * r2
                # Equation (3.4)

                D_delta = abs(C3 * self.Delta_pos[j] - self.Positions[i, j])
                # Equation (3.5)-part 3
                X3 = self.Delta_pos[j] - A3 * D_delta
                # Equation (3.5)-part 3

                self.Positions[i, j] = (X1 + X2 + X3) / 3  # Equation (3.7)
        return
    
     def Evaluate(self):
        self.gerar_populacao()
        for l in range(0, self.max_ite):
            
            self.calculateFitness()
            a =  2 - l * ((2) / self.max_ite)
            self.updateWolves(a)
            self.hist.append(self.Alpha_score)
           # if l % 1 == 0:
               # print(["At iteration " + str(l) + " the best fitness is " + str(self.Alpha_score)])
        return
    
  

        

In [ ]:
# Call instance of GWO
dimensions = (n_inputs * n_hidden) + (n_hidden * n_hidden)+(n_hidden*n_classes) + 2*n_hidden + n_classes
max_ite = 500
n_agents = 30
lb = [-2]*dimensions
up = [2]*dimensions
hist_sim=[]
best_sim = []
t1 = time.time()

for i in range(10):
    print("Simulação " + str(i))
    gwo = GWO(n_agents,max_ite,dimensions,lb,up)
    gwo.Evaluate()
    hist_sim.append(gwo.hist)
    best_sim.append(gwo.Alpha_pos)
tempoExec = time.time() - t1
print("Tempo de execução: {} segundos".format(tempoExec))

Simulação 0


In [ ]:
def predict(pos):
    """
    Use the trained weights to perform class predictions.

    Inputs
    ------
    pos: numpy.ndarray
        Position matrix found by the swarm. Will be rolled
        into weights and biases.
    """
    logits = logits_function(pos,X_test)
    
    y_pred = np.argmax(logits, axis=1)
   
    return y_pred

In [ ]:
numpy_array = np.array(hist_sim[:])
transpose = numpy_array.T
transpose_list = transpose.tolist()
convergence  = pd.DataFrame(transpose_list)
convergence

In [ ]:
best = []
for i, column in enumerate(convergence.iloc[:, :]):
   ax = convergence[column].plot.line(x='pig', y='horse',title='Curvas de convergência')
   ax.set_xlabel("Iterações")
   ax.set_ylabel("Função Objetivo")
   #melhores resultados de cada simulação
   best.append(min(convergence[column]))

In [ ]:
min_ind = np.argmin(best)
best[min_ind]

In [ ]:

(predict(best_sim[min_ind]) == y_test).mean()

In [ ]:
BestGWO  = pd.DataFrame(best,columns =['BestGWO'])
BestGWO
BestGWO.to_excel("BestGWO.xlsx")

In [ ]:
BestGWO_POS  = pd.DataFrame(best_sim)

BestGWO_POS.to_excel("BestGWO_POS.xlsx")